# Cycle GAN

This is a raw version containing copied passages from the 8th homework implementing a generative adversarial network


## 0. Import required packages

In [1]:
# import relevant libraries
import numpy as np
import tensorflow as tf
import struct
import matplotlib.pyplot as plt

## 1. The Data


### 1.1 Download the MNIST and the SVHN dataset

### 1.2 Read in the training images

In [ ]:
# importing mnist
def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)
    
data = read_idx('../MNIST/train-images.idx3-ubyte')
training_labels = read_idx('../MNIST/train-labels.idx1-ubyte')


### not done yet. 

# 2. Build the model

## 2.1 Helper functions for leyers

Following cells contain helper functions to create different leyers of our model. (JUST COPIED FROM OLD HOMEWORK (8).

In [ ]:
# helper functions
# not sure which we will use and maybe we have to change some

def feed_forward_layer(x, hidden_n, activation_fn, normalize):
    initializer = tf.random_normal_initializer(stddev=0.02)
    weights = tf.get_variable("weights", [x.shape[1], hidden_n], tf.float32, initializer)
    biases = tf.get_variable("biases", [hidden_n], tf.float32, tf.zeros_initializer())
   
    drive = tf.matmul(x, weights) + biases
    if normalize:
        drive = batch_norm(drive, [0])
   
    if activation_fn == 'linear':
        return drive
    else:
        return activation_fn(drive)

In [ ]:
def conv_layer(x, kernels_n, kernel_size, stride_size, activation_fn, normalize):
    initializer = tf.random_normal_initializer(stddev=0.02)
    kernels = tf.get_variable("kernels", [kernel_size, kernel_size, x.shape[-1], kernels_n], tf.float32, initializer)
    biases = tf.get_variable("biases", [kernels_n], tf.float32, tf.zeros_initializer())

    drive = tf.nn.conv2d(x, kernels, strides = [1, stride_size, stride_size, 1], padding = "SAME") + biases
    if normalize:
        drive = batch_norm(drive, [0,1,2])
    
    return activation_fn(drive)

In [ ]:
def back_conv_layer(x, target_shape, kernel_size, stride_size, activation_fn, normalize):
    initializer = tf.random_normal_initializer(stddev=0.02)
    kernels = tf.get_variable("kernels", [kernel_size, kernel_size, target_shape[-1], x.shape[-1]], tf.float32, initializer)
    biases = tf.get_variable("biases", [target_shape[-1]], tf.float32, tf.zeros_initializer())

    drive = tf.nn.conv2d_transpose(x, kernels, target_shape, strides = [1, stride_size, stride_size, 1], padding = "SAME") + biases
    if normalize:
        drive = batch_norm(drive, [0,1,2])
    
    return activation_fn(drive)

In [ ]:
def flatten(x):
    size = int(np.prod(x.shape[1:]))
    return tf.reshape(x, [-1, size])



In [ ]:
def batch_norm(x, axes):
    mean, var = tf.nn.moments(x, axes = axes)
    offset_initializer = tf.constant_initializer(0.0)
    offset = tf.get_variable("offset", [x.shape[-1]], tf.float32, offset_initializer)
    scale_initializer = tf.constant_initializer(1.0)
    scale = tf.get_variable("scale", [x.shape[-1]], tf.float32, scale_initializer)
    return tf.nn.batch_normalization(x, mean, var, offset, scale, 1e-6)

### 2.2 Generator
The generator creates new images.
#### 2.2.1 Input to generator
As an input for our generator xy vectors with xy dimensions are used. Because we have o feed in these vectors we need to define a placeholder. 

In [3]:
# Before we can create our graph, we need to reset the graph
tf.reset_default_graph()

# Create the placeholder for the generator input
generator_input = tf.placeholder(tf.float32, shape=(32, 50))

#### 2.2.2 Generator Layer 1
The first layer of the generator is a feed forward layer with XY hidden neurons, XY activation function and batch normalization(?). After the feed forward step the output is reshaped to XY feature maps of size XxY. 

In [ ]:
with tf.variable_scope('generator_layer_1', reuse=tf.AUTO_REUSE):
    gen_output_unshaped = feed_forward_layer(x=generator_input, 
                                             hidden_n=1024, 
                                             activation_fn=tf.nn.relu, 
                                             normalize=True)
    gen_output = tf.reshape(gen_output_unshaped, [32, 4, 4, 64])

In [ ]:
####followingly define different leyers for the generater, e.g.

with tf.variable_scope('generator_layer_2', reuse=tf.AUTO_REUSE):
    gen_output = back_conv_layer(x=gen_output, 
                                  target_shape=[32, 7, 7, 32], 
                                  kernel_size=5, 
                                  stride_size=2, 
                                  activation_fn=tf.nn.relu, 
                                  normalize=True)
    
with tf.variable_scope('gengenerator_layer_3', reuse=tf.AUTO_REUSE):
    gen_output = back_conv_layer(x=gen_output, 
                                  target_shape=[32, 14, 14, 16], 
                                  kernel_size=5, 
                                  stride_size=2, 
                                  activation_fn=tf.nn.relu, 
                                  normalize=True)

### 2.3 Discriminator
The discriminator wants to find out which images are original and which are generated.
#### 2.3.1 Input to Discriminator
The input for the discriminator is the XY previously generated images and XY original images. 

In [ ]:
# Create a dataset from the MNIST training images
dataset = tf.data.Dataset.from_tensor_slices(data)

# Shuffle the dataset
dataset = dataset.shuffle(buffer_size=4, reshuffle_each_iteration=True)

# Define the batch
dataset = dataset.batch(32)

# Create the iterator and get the next batch
iterator = tf.data.Iterator.from_structure(dataset.output_types,
                                           dataset.output_shapes)
next_batch = iterator.get_next()

# initializer for MNIST dataset
init_op = iterator.make_initializer(dataset)

MNIST_images = tf.cast(next_batch, tf.float32)

# Concatinate the original MNIST training images and the generated images
discriminator_input = tf.concat([gen_output, tf.expand_dims(MNIST_images, axis=-1)], axis=0)

#### 2.3.2 Discriminator Layer 1 & 2 & 3
The first three layers of the discriminator are normal convolutional layers with respectively 8, 16 and 32 feature maps. All use leaky ReLU activation function and batch normalization.

In [ ]:
with tf.variable_scope('discriminator_layer_1', reuse=tf.AUTO_REUSE):
    disc_output = conv_layer(x=discriminator_input, 
                              kernels_n=8, 
                              kernel_size=5, 
                              stride_size=2, 
                              activation_fn=tf.nn.leaky_relu, 
                              normalize=True)

with tf.variable_scope('discriminator_layer_2', reuse=tf.AUTO_REUSE):
    disc_output = conv_layer(x=disc_output, 
                              kernels_n=16, 
                              kernel_size=5, 
                              stride_size=2, 
                              activation_fn=tf.nn.leaky_relu, 
                              normalize=True)
    
with tf.variable_scope('discriminator_layer_3', reuse=tf.AUTO_REUSE):
    disc_output = conv_layer(x=disc_output, 
                              kernels_n=32, 
                              kernel_size=5, 
                              stride_size=2, 
                              activation_fn=tf.nn.leaky_relu, 
                              normalize=True)
    
    
    ####in the hw a fourth leyer was implemented

### 2.4 Training
We want to train the generator and the discriminator seperatly.
#### 2.4.1 Training the Generator
Because we want to fool the discriminator with our generated images, we need 32 ones as labels. We calculate the loss by taking the mean of the sigmoid cross entropy between the labels (32 ones) and the first half of the discriminators output (the logits that correspond to the generated images). Then we take an Adam optimizer to minimize the loss and only change the variables of our generator. 

In [ ]:
# We need a list with 32 ones as labels
generator_training_labels = tf.ones([32, 1])

# Take the first 32 outputs of the discriminator
generator_training_outputs = tf.slice(disc_output, [0, 0], [32, 1])

# Calculate the loss
generator_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=generator_training_labels, logits=generator_training_outputs))

# We just want to train the generator 
trainable_variables = tf.trainable_variables()
generator_variables = [var for var in trainable_variables if 'generator' in var.name]
generator_optimizer = tf.train.AdamOptimizer(learning_rate=0.0004, beta1=0.5)
training_step_generator = generator_optimizer.minimize(generator_loss, var_list=generator_variables)

#### 2.4.2 Training the Discriminator
For the training of the discriminator we need 64 labels. 32 ones for the generated images and 32 ones for the original images. Then we calculate the loss and optimize all variables of the discriminator with an Adam optimizer. 

In [ ]:
# We need a list with 32 zeros and a list with 32 ones as labels
discriminator_training_labels = tf.concat([tf.zeros([32, 1]), tf.ones([32,1])], axis=0)

# Calculate the loss
discriminator_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=discriminator_training_labels, logits=disc_output))

# We just want to train the discriminator
trainable_variables = tf.trainable_variables()
discriminator_variables = [var for var in trainable_variables if 'discriminator' in var.name]
discriminator_optimizer = tf.train.AdamOptimizer(learning_rate=0.0004, beta1=0.5)
training_step_discriminator = discriminator_optimizer.minimize(discriminator_loss, var_list=discriminator_variables)

## 3 Training
### 3.1 Monitoring
To visualize the losses and the generated images we use TensorBoard. Thus, we create scalar summaries for the losses and an image summary for the generated images. Furthermore, we define two filewriter for the training and validation.

In [ ]:
# Add summaries for monitoring the loss
generator_loss_summary = tf.summary.scalar('generator-loss', generator_loss)
discriminator_loss_summary = tf.summary.scalar('discriminator-loss', discriminator_loss)

# Merge loss summaries
loss_summary = tf.summary.merge([generator_loss_summary, discriminator_loss_summary])

# Add summary for visualizing the generated images
generated_images_summary = tf.summary.image('generated-images', gen_output, max_outputs=32)

# Define the filewriter for storing the values
train_writer = tf.summary.FileWriter(logdir='./summaries/train/')
validation_writer = tf.summary.FileWriter(logdir='./summaries/validation/', flush_secs=2)

### 3.2 Training and validation
We train the model for 2 epochs. In each step we first sample 32 random vectors with 50 dimensions from an uniform distribution. This vectors are than feed in the generator during the training. We then run both training steps (for the generator andd the discriminator) and read out and save the loss summaries. After each 100th step we validate the model. Thus, we feed in the vector sampled at the beginning for the validation and read out the summaries for the generated images.  

In [ ]:
# define training epoches
epochs = 2

# start the training
with tf.Session() as sess:
    
    # initialize the variables for the session
    sess.run(tf.global_variables_initializer())
    
    # initialize step counter for saving the summaries
    global_step = 1
    
    # create the random vector for the validation
    validation_input = np.random.uniform(-1, 1, [32, 50])
    
    for epoch in range(epochs):
        # load the training data into the iterator
        sess.run(init_op)
        
        while True:
            try:
                # create the random vector for the generator
                random_input = np.random.uniform(-1, 1, [32, 50])
                
                # train the generator and discriminator andd read out the loss summaries for the generator and discriminator
                # unfortunately, we cannot use the merged summary because this results in the a error we were not able to solve
                # error: Fetch argument b'\n\x10\n\tgenerator-loss\x15\xe8TV?\n\x14\n\rdiscriminator-loss\x15\xc2\xd5/?' cannot be interpreted as a Tensor. ('utf-8' codec can't decode byte 0xe8 in position 19: invalid continuation byte)
                # _, _, gen_loss_summary, disc_loss_summary  = sess.run((training_step_generator, training_step_discriminator, generator_loss_summary, discriminator_loss_summary), 
                #                           feed_dict={generator_input: random_input})
                _, _, loss_summaries  = sess.run((training_step_generator, training_step_discriminator, loss_summary), 
                                          feed_dict={generator_input: random_input})
                # write the loss summaries
                # train_writer.add_summary(gen_loss_summary, global_step)
                # train_writer.add_summary(disc_loss_summary, global_step)
                train_writer.add_summary(loss_summaries, global_step)
                
                # read out the generated images
                if (global_step % 100 == 0):
                    image_summary = sess.run((generated_images_summary), feed_dict={generator_input: validation_input})
                    validation_writer.add_summary(image_summary, global_step)
                
                # update the step counter
                global_step += 1
                
            # breakout of the loop if we looked at all batches and finished our training
            except tf.errors.OutOfRangeError:
                break

### 3.3 Results